In [1]:
import os
import sys
import pandas as pd
import numpy as np
import time
import warnings

from tqdm import tqdm

warnings.filterwarnings("ignore")

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


sys.path.append("..")

from utils import DATA_DIR  # noqa

In [3]:
# BertのモデルとTokenizer(前処理用)をimport
from transformers import BertTokenizer, BertModel

In [4]:
class BertClf(nn.Module):
    def __init__(self, input_size=768, num_classes=3, dropout_rate=0.3):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)

        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        output = self.dropout(x)
        output = self.linear(output)
        proba = F.softmax(output, dim=1)

        return output, proba

In [5]:
tweet_df = pd.read_csv(os.path.join(DATA_DIR, "cleaned_airline_tweets.csv"))
tweet_df["sentiment"] = tweet_df["sentiment"].replace({"negative": 0, "neutral": 1, "positive": 2})

train, test = train_test_split(tweet_df, test_size=0.2, random_state=0, stratify=tweet_df["sentiment"])
train, test = train.reset_index(drop=True), test.reset_index(drop=True)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else 
                      "mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [7]:
bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [8]:
A = ["hello my name is chu. Nice to meet you"]
C = tokenizer(A)
for c in C:
    print(f"{c}: {C[c]}")

input_ids: [[101, 7592, 2026, 2171, 2003, 14684, 1012, 3835, 2000, 3113, 2017, 102]]
token_type_ids: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


In [9]:
train_dataloader = DataLoader(
    train["text"],
    batch_size=256,
    shuffle=False,
    collate_fn=lambda batch: tokenizer(
        text=batch,
        padding="longest",
        truncation=True,
        return_tensors="pt",
        max_length=128
    )
)

test_dataloader = DataLoader(
    test["text"],
    batch_size=256,
    shuffle=False,
    collate_fn=lambda batch: tokenizer(
        text=batch,
        padding="longest",
        truncation=True,
        return_tensors="pt",
        max_length=128
    )
)

In [12]:
train_emb_list = []
test_emb_list = []
bert_model.eval()
with torch.no_grad():
    for batch in tqdm(train_dataloader):
        outputs = bert_model(
            input_ids=batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            token_type_ids=batch["token_type_ids"].to(device)
        )
        embedding = outputs.pooler_output
        train_emb_list.append(embedding)
        torch.mps.empty_cache()

train_emb = torch.vstack(train_emb_list)
train_emb_label = torch.tensor(train["sentiment"]).to(device)
train_dataset = TensorDataset(train_emb, train_emb_label)
train_loader = DataLoader(train_dataset, batch_size=len(train_dataset) // 10, shuffle=True)


test_emb_list = []
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        outputs = bert_model(
            input_ids=batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            token_type_ids=batch["token_type_ids"].to(device)
        )
        embedding = outputs.pooler_output
        test_emb_list.append(embedding)
        torch.mps.empty_cache()

test_emb = torch.vstack(test_emb_list)
test_emb_label = torch.tensor(test["sentiment"])

100%|██████████| 4/4 [00:01<00:00,  2.14it/s]


In [14]:
bert_clf = BertClf().to(device)
optimizer = optim.Adam(bert_clf.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

num_epochs = 1000
bert_clf.train()
for epoch in tqdm(range(num_epochs), "Traning Progress"):
    total_loss = 0
    num_batches = 0

    for batch_emb, batch_labels in train_loader:
        batch_emb = batch_emb.to(device)
        batch_labels = batch_labels.to(device)

        optimizer.zero_grad()

        output, proba = bert_clf(batch_emb)
        loss = criterion(output, batch_labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

    avg_loss = total_loss / num_batches
        
        

Traning Progress: 100%|██████████| 1000/1000 [00:53<00:00, 18.81it/s]


In [15]:
bert_clf.eval()
with torch.no_grad():
    train_output, train_proba = bert_clf(train_emb)
    train_pred = torch.argmax(train_proba, dim=1).cpu().numpy()

    test_output, test_proba = bert_clf(test_emb)
    test_pred = torch.argmax(test_proba, dim=1).cpu().numpy()

print("\n=== Training Set Results ===")
print(classification_report(train["sentiment"].values, train_pred, 
                           target_names=["negative", "neutral", "positive"]))

print("\n=== Test Set Results ===")
print(classification_report(test["sentiment"].values, test_pred, 
                           target_names=["negative", "neutral", "positive"]))


=== Training Set Results ===
              precision    recall  f1-score   support

    negative       0.84      0.72      0.77       972
     neutral       0.71      0.80      0.75      1039
    positive       0.85      0.84      0.85      1077

    accuracy                           0.79      3088
   macro avg       0.80      0.79      0.79      3088
weighted avg       0.80      0.79      0.79      3088


=== Test Set Results ===
              precision    recall  f1-score   support

    negative       0.87      0.73      0.79       243
     neutral       0.73      0.84      0.78       260
    positive       0.84      0.83      0.84       269

    accuracy                           0.80       772
   macro avg       0.81      0.80      0.80       772
weighted avg       0.81      0.80      0.80       772



In [16]:
from sklearn.linear_model import LogisticRegression

train_emb_cpu = train_emb.cpu().numpy()
test_emb_cpu = test_emb.cpu().numpy()
# 比較用：scikit-learnのLogisticRegression
logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg.fit(train_emb_cpu, train["sentiment"])
print("\n=== Comparison: Scikit-learn LogisticRegression ===")
print(classification_report(test["sentiment"].values, logreg.predict(test_emb_cpu), target_names=["negative", "neutral", "positive"]))


=== Comparison: Scikit-learn LogisticRegression ===
              precision    recall  f1-score   support

    negative       0.85      0.86      0.85       243
     neutral       0.81      0.82      0.82       260
    positive       0.88      0.86      0.87       269

    accuracy                           0.85       772
   macro avg       0.85      0.85      0.85       772
weighted avg       0.85      0.85      0.85       772

